In [2]:
import pandas as pd
#from sklearn_pandas import DataFrameMapper
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

/Users/VyUng/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error  
from sklearn.metrics import r2_score,accuracy_score
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

In [4]:
from sklearn.cross_validation import train_test_split

In [5]:
from sklearn.cross_validation import cross_val_score

In [6]:
accept=pd.read_csv('/Users/VyUng/Desktop/LoanStats3a.csv'
                   ,low_memory=False)

In [7]:
accept['result']=[1]*len(accept)

In [8]:
reject=pd.read_csv('/Users/VyUng/Desktop/RejectStatsA.csv'
                   ,low_memory=False)

In [9]:
reject['result']=[0]*len(reject)

In [10]:
request = list(reject.request) + list(accept.loan_amnt)

In [11]:
dti = list(reject.dti) + list(accept.dti)

In [12]:
state = list(reject.State) + list(accept.addr_state)

In [13]:
emp_length = list(reject.emp_length) + list(accept.emp_length)

In [14]:
funded_amnt = [0]*len(reject) + list(accept.funded_amnt)

In [15]:
result = list(reject.result)+ list(accept.result)

In [16]:
data = pd.DataFrame.from_items([('request',request), 
                                  ('dti',dti),
                                  ('state',state),
                                  ('emp_length',emp_length),
                                  ('funded_amnt', funded_amnt),
                                ('result', result)])

In [17]:
data.sample(1)

,request,dti,state,emp_length,funded_amnt,result
328220,1500.0,0.0,MD,< 1 year,0,0


In [18]:
#data.isnull().sum()
data = data.dropna()
emp1 =['< 1 year','1 year']
emp2 =['2 years','3 years','4 years','5 years']
emp3 =['6 years','7 years','8 years','10 years']
emp4 =['10+ years']
emp=[]

In [19]:
for index, row in data.iterrows():
    if row['emp_length'] in emp1:
        emp.append(1)
    elif row['emp_length'] in emp2:
        emp.append(2)
    elif row['emp_length'] in emp3:
        emp.append(3)
    else:
        emp.append(4)
data['emp']=emp
        

In [20]:
data.sample(3)

,request,dti,state,emp_length,funded_amnt,result,emp
597940,2000.0,0.00,NY,< 1 year,0,0,1
599944,35000.0,0.00,OH,< 1 year,0,0,1
236734,3900.0,19.85,PA,< 1 year,0,0,1


In [21]:
#Divide states by regions based on the "standard federal region"
#https://en.wikipedia.org/wiki/List_of_regions_of_the_United_States
region1=['CT', 'ME', 'MA', 'NH', 'RI', 'VT']
region2=['NJ', 'NY', 'PR', 'VI'] 
region3=['DE', 'MD', 'PA', 'VA', 'WV'] 
region4=['AL', 'FL', 'GA', 'KY', 'MS', 'NC', 'SC', 'TN'] 
region5=['IL', 'IN', 'MI', 'MN', 'OH', 'WI']
region6=['AR', 'LA', 'NM', 'OK', 'TX']
region7=['IA', 'KS', 'MO', 'NE']
region8=['CO', 'MT', 'ND', 'SD', 'UT', 'WY']
region9=['AZ', 'CA', 'HI', 'NV', 'GU']
region10=['AK', 'ID', 'OR', 'WA'] 
region=[]

In [22]:
for index, row in data.iterrows():
    if row['state'] in region1:
        region.append(1)
    elif row['state'] in region2:
        region.append(2)
    elif row['state'] in region3:
        region.append(3)
    elif row['state'] in region4:
        region.append(4)
    elif row['state'] in region5:
        region.append(5)
    elif row['state'] in region6:
        region.append(6)
    elif row['state'] in region7:
        region.append(7)
    elif row['state'] in region8:
        region.append(8)
    elif row['state'] in region9:
        region.append(9)
    elif row['state'] in region10:
        region.append(10)
    else:
        region.append(5)


In [23]:
data['region']=region

In [24]:
data.sample(3)

,request,dti,state,emp_length,funded_amnt,result,emp,region
370793,35000.0,5.65,CA,< 1 year,0,0,1,9
400976,15000.0,9.51,FL,9 years,0,0,4,4
770973,16000.0,22.08,WA,4 years,16000,1,2,10


In [25]:
x = np.array(data[['request','dti','emp','region']]).reshape(-1,4)
y = data['result']

In [26]:
#Model evaluation using a validation set:
(x_train,x_test,y_train,y_test)=train_test_split(x,y,test_size=0.75,
                                                 random_state=42)

In [27]:
model1 = LogisticRegression()
model1.fit(x_train, y_train)
predicted=model1.predict(x_test)
print predicted

[0 0 0 ..., 0 0 0]


In [28]:
#Generate class probabilities
#
probs = model1.predict_proba(x_test)
print probs

[[ 0.95543267  0.04456733]
 [ 0.96392641  0.03607359]
 [ 0.86747152  0.13252848]
 ..., 
 [ 0.99161918  0.00838082]
 [ 0.97738763  0.02261237]
 [ 0.98027078  0.01972922]]


In [29]:

#Generate evaluation matrics 
print accuracy_score(y_test,predicted)
print roc_auc_score(y_test,probs[:,1])
print confusion_matrix(y_test, predicted)
print classification_report(y_test, predicted)

0.964493535767
0.695483776596
[[566530      0]
 [ 20856      0]]
             precision    recall  f1-score   support

          0       0.96      1.00      0.98    566530
          1       0.00      0.00      0.00     20856

avg / total       0.93      0.96      0.95    587386



/Users/VyUng/anaconda2/lib/python2.7/site-packages/sklearn/metrics/metrics.py:1905: UserWarning: The sum of true positives and false positives are equal to zero for some labels. Precision is ill defined for those labels [1]. The precision and recall are equal to zero for some labels. fbeta_score is ill defined for those labels [1]. 
  average=None)


In [30]:
#Model evaluation using cross-validation:
scores = cross_val_score(LogisticRegression(),x,y,
                         scoring='accuracy',cv=10)
print scores
print scores.mean()

[ 0.96460629  0.96456753  0.9645803   0.96461861  0.964542    0.96461861
  0.96461861  0.96461861  0.96460584  0.96460584]
0.964598221857


In [31]:
scores2 = cross_val_score(LogisticRegression(),x,y,
                          scoring='roc_auc',cv=10)
print scores2
print scores2.mean()

[ 0.69791394  0.6872579   0.68838829  0.68695926  0.66462151  0.81394411
  0.80514369  0.81198556  0.67624401  0.67586043]
0.720831869525
